## Oppgave 1

$K = 1001$

$ipad = 0011$

$opad = 0101$

$h(x) = x^2\pmod{2^8}$ hvor vi henter ut kun de 4 midterste tallene i 8-bitstrengen. (Kan gjøres med en AND operasjon med tallet 00111100, etterfulgt av 2 bitskift til høyre. Se demo i python kode celle under.)

$HMAC(K,x) = h((K \oplus opad)||h((K \oplus ipad)||x))$

### a)
melding her omtales som x:
x = 0110

Finner $HMAC(1001, 0110)$:
- $K \oplus ipad = 1001 \oplus 0011 = 1010$
- $(K \oplus ipad) || x = 1010 || 0110 = 10100110$
- $h((K \oplus ipad)||x) = h(10100110) = ((10100110^2\pmod{2^8})\ \&\ 00111100) >> 2 = 1001$
- $K \oplus opad = 1001 \oplus 0101 = 1100$
- $(K \oplus opad)||h((K \oplus ipad)||x) = 1100||1001 = 11001001$
- $h((K \oplus opad)||h((K \oplus ipad)||x)) = ((11001001^2\pmod{2^8})\ \&\ 00111100) >> 2 = 0100$

$HMAC(1001, 0110) = \underline{\underline{0100}}$

### b)

Ja fordi vi vet at HMAC 0100 skal tilsvare melding 0110. Altså er det grunn til å tro at meldingen ikke er autentisk.

In [1]:

bin(((int('0b10100110',2)**2 % 2**8) & int('0b00111100', 2)) >> 2)

'0b1001'

In [2]:
bin(((int('0b11001001',2)**2 % 2**8) & int('0b00111100', 2)) >> 2)

'0b100'

In [3]:
## Oppgave 2

# Blocksize is predifined as 4-bit
BLOCK_SIZE = 2**4

def cesar(x):
    return (x+3) % BLOCK_SIZE

def cbc_mac(msg):
    y = 0
    prev_y = 0
    for x in msg:
        y = cesar(prev_y ^ x)
        prev_y = y
    return y

print(f"CBC-MAC(x ): {bin(cbc_mac([int('1101',2), int('1111',2), int('1010',2), int('0001',2)]))}")
print(f"CBC-MAC(x'): {bin(cbc_mac([int('0010',2), int('1100',2), int('0001',2), int('1111',2)]))}")

CBC-MAC(x ): 0b1101
CBC-MAC(x'): 0b10


## Oppgave 3

### a)

$z_{i+4} = z_{i} + z_{i+1} + z_{i+2} + z_{i+3}\pmod 2$

Nøkler:
- $K_{1} = 1000$
- $K_{2} = 0011$
- $K_{3} = 1111$

m = 4

Vi har kun nøkler 1 til 3 som betyr at vi ikke kjenner $z_{i+3}$

Finner $z_{i+3}$:
- $z_{i+3} = z_{i} + z_{i+1} + z_{i+2}\pmod 2 = c_{1} \cdot K_{1} + c_{2} \cdot K_{2} + c_{3} \cdot K_{3} \pmod 2 = 1 \cdot 1000 + 1 \cdot 0011 + 1 \cdot 1111 \pmod 2$


Dette gir en maks periode på $2^{4} - 1 = \underline{\underline{15}}$

### b)
Vi har samme m her (m=4)

Dermed vil også maks perioden her være $2^{4} - 1 = \underline{\underline{15}}$

In [12]:
z_3 = (int('0b1000', 2) + int('0b0011', 2) + int('0b1111', 2)) % 2
print(z_3)

z_4 = (z_3 + z_3) % 2
print(z_4)

0
0


In [ ]:
z_3 = (int('0b1000', 2) + int('0b0011', 2) + int('0b1111', 2)) % 2
print(z_3)

z_4 = (z_3 + z_3) % 2
print(z_4)

## Oppgave 4

### a)

*Se kode celle under*

### b)

Kriterier for valg av p og q:
- p og q skal begge være primtall fra og med 256, til og med 65535 (heltall representert med 2 og kun 2 bytes).
- Siden $e = 3$ er gitt så er valget av p og q basert på kompatibilitet med denne variabelen. Mer spesifikt så må den største felles faktoren mellom e og LCM(p-1, q-1) være 1.


### c)

In [50]:
## Oppgave 4
import sympy, math

PRIME_RANGE_LOWER_BOUND = 256
PRIME_RANGE_UPPER_BOUND = 65535

#source: https://www.geeksforgeeks.org/euclidean-algorithms-basic-and-extended/
def gcd_extended(a, b):
    if a == 0:
        return b, 0, 1
    
    d, x_tmp, y_tmp = gcd_extended(b % a, a)

    x = y_tmp - (b // a) * x_tmp
    y = x_tmp

    return d, x, y

# source: https://www.geeksforgeeks.org/exponential-squaring-fast-modulo-multiplication/
def exponentiation(bas, exp, N):
    t = 1
    while(exp > 0):
        # for cases where exponent
        # is not an even value
        if (exp % 2 != 0):
            t = (t * bas) % N;
        bas = (bas * bas) % N;
        exp = int(exp / 2);
    return t % N;

def key_gen():
    l_n = -1
    is_not_done = True
    
    # Public key part
    e = 3
    
    while is_not_done:
        p = sympy.randprime(PRIME_RANGE_LOWER_BOUND, PRIME_RANGE_UPPER_BOUND)
        q = sympy.randprime(PRIME_RANGE_LOWER_BOUND, PRIME_RANGE_UPPER_BOUND)
        while p == q: #Ensure p and q arent equal
            q = sympy.randprime(PRIME_RANGE_LOWER_BOUND, PRIME_RANGE_UPPER_BOUND)

        # Public key part
        n = p * q

        l_n = ((p-1) * (q-1)) // math.gcd(p-1,q-1)

        if math.gcd(e, l_n) == 1:
            is_not_done = False

    _, x, y = gcd_extended(e, l_n)

    d = -1
    #Ensure we pick the biggest value
    if x > y and (x * e) % l_n == 1:
        d = x
    elif (y * e) % l_n == 1:
        d = y
    else:
        n, e, d = key_gen() ## Try again

    return n, e, d

def encrypt(msg, e, n):
    return exponentiation(msg, e, n)

def decrypt(cipher_text, d, n):
    return exponentiation(cipher_text, d, n)

n, e, d = key_gen()

#public_key  = (n, e)
#private_key = (n, d)

start_value = 12345
print(f"Start text: {start_value}")

encrypted_value = encrypt(start_value, e, n)
print(f"Cipher text: {encrypted_value}")

end_value = decrypt(encrypted_value, d, n)
print(f"End text: {end_value}")

Start text: 12345
Cipher text: 3008742485
End text: 12345


## Oppgave 5

### a)

Vi starter med:

$n = 1829$

$B = 5$

Velger $a$ (en faktor av n): 

$a = 2$

Regner ut A:

$A = a^{B!}\pmod{n} = 2^{5!}\pmod{1829} = 2^{120}\pmod{1829} = 311$

Regner ut F:

$F = gcd(A-1,n) = gcd(310, 1829) = 31$

Siden F er større enn 1 vet vi at F er en primtalls faktor av n

Deler n på F:

$E = n / F = 1829 / 31 = 59$

Dermed har vi funnet to primtalls faktorer $\underline{\underline{31}}$ og $\underline{\underline{59}}$ som sammen danner $1829$ ($31 \cdot 59 = 1829$).

### b)

Fra 

### c)
$n = 6319$

In [51]:
import math
## Oppgave 5

def pollard(n, B):
    a = 2

    for i in range(2,B):
        a = (a**i) % n
    print(a)
    d = math.gcd(a-1, n)
    print(d)
    if 1 < d < n:
        return d
    
    return "failed"

n = 1829
B = 5
a = 2

A = a**math.factorial(B) % n
print(A)
F = math.gcd(A-1, n)

E = n / F

pollard(n, B)

311
1628
1


'failed'

## Oppgave 6

### a)

$VS$: $e_{K}(x_{1}) \cdot e_{K}(x_{2}) \mod n$

$HS$: $e_{K}(x_{1} \cdot x_{2}) \mod n$

Funksjonen $e_{K}(x)$ tar $x$ som input og gir $x^{e}$ som output

Vi får da:

$VS$: $e_{K}(x_{1}) \cdot e_{K}(x_{2}) \mod n = (x_{1}^{e}) \cdot (x_{2}^{e}) \mod n$

Bruker potensregelen for å trekke ut $e \implies VS = (x_{1} \cdot x_{2})^{e} \mod n$

$HS$: $e_{K}(x_{1} \cdot x_{2}) \mod n = (x_{1} \cdot x_{2})^{e} \mod n$

$VS = HS$

$QED$

### b)


- $x' = d_{K}(y')$
- $ = d_{K}(y \cdot r^{e})^{d}$
- $ = (y \cdot r^{e})^{d}$
- $ = (e_{K}(x) \cdot r^{e})^{d}$
- $ = (x^{e} \cdot r^{e})^{d}$
- $ = (x^{e})^{d} \cdot (r^{e})^d \mod n$
- $ = x \cdot r \mod n$

$ \implies x = x' \cdot r^{-1} \mod n = x' / r \mod n$

## Oppgave 7

### a)

Det er gitt at $p$ og $q$ er begge primtall og oddetall større enn $2$

Vi kan skrive $q - p = 2d$, der $d$ er et heltall, fordi $q$ og $p$ er oddetall. 

Vi kan først si at $q - p = D$ (et heltall og partall). Oddetall + eller minus et annet oddetall gir alltid et partall, med $0$ som unntak. 

Siden $D$ er partall kan vi dele det på $2$ slik: $D / 2 = d$. 

Vi har nå et annet heltall (kan være enten partall eller oddetall). Dette kan vi skrive om igjen til $D = 2d$ som da betyr at $q - p = 2d$. 

### b)

$n = q \cdot p$

$q - p = 2d \implies d = (q - p) / 2$

- $n + d^{2} = (q \cdot p) + ((q - p) / 2)^{2}$
- $= q \cdot p + (q - p)^{2} / 4$
- $= q \cdot p + (q^{2} - 2qp + p^{2}) / 4$
- $= (4 \cdot (q \cdot p)) / 4 + (q^{2} - 2qp + p^{2}) / 4$
- $= (4 \cdot (q \cdot p) + q^{2} - 2qp + p^{2}) / 4$
- $= (q^{2} + 2qp + p^{2}) / 4$
- $= ((q + p)^{2}) / 4$
- $= ((q + p)^{2}) / 2^{2}$
- $= ((q + p) / 2)^{2}$

$d = (q - p) / 2$ og $t = (q + p) / 2$

$n + d^{2} = t^{2}$

Vi ser at $n + d^{2}$ er et kvadrat (altså kvadrattall) av tallet $(q + p) / 2$

### c)

- $t^{2} = n + d^{2}$
- $\implies n = t^{2} - d^{2}$
- $\implies q \cdot p = t^{2} - d^{2}$
- $\implies q \cdot p = (t - d) \cdot (t + d)$
- $\implies q = t - d$ og $p = t + d$


Anntar $d^{2}$ er liten nok.

- $\implies t = \sqrt{n + d^{2}} \approx \sqrt{n}$

Vi kan prøve oss fram med å øke t med en av gangen til vi får en d ($\sqrt{t^{2}-n}$) som et heltall

Da får vi at $q = t - \sqrt{t^{2}-n}$ og $p = t + \sqrt{t^{2}-n}$, for $\sqrt{t^{2}-n} \in \mathbb{Z}$

### d)

$n = 152416580095517$

$t \approx \sqrt{n} = \sqrt{152416580095517} = $

In [6]:
def factorize_to_close_primes(n):
    q, p = -1, -1
    notDone = True
    i = 0
    while notDone:
        t = math.ceil(math.sqrt(n)) + i
        d = math.sqrt(t**2 - n)
        if d.is_integer():
            q = t - d
            p = t + d
            notDone = False
        
        i += 1
            
    return q, p
        
factorize_to_close_primes(152416580095517)

(12345709.0, 12345713.0)

In [7]:
12345709.0 * 12345713.0

152416580095517.0